In [8]:
# coding: utf-8

# In[62]:

from Levenshtein import distance


# In[68]:

import numpy as np

def get_levenshtein(x,y):
    a=max(len(x), len(y))
    b=min(len(x), len(y))
    if a==0:
        return 100
    if a > b+500:
        return 100
    return 2 * distance(x,y)/(len(x) + len(y))

In [7]:
import pandas as pd
import os
import unicodedata

def normalize_text(x):
    return unicodedata.normalize("NFKD", x.replace(";"," "))

projet_loi = normalize_text("EQUILIBRE DANS LE SECTEUR AGRICOLE ET ALIMENTAIRE")


def lecture_amendement(folder):
    res=""
    starts=[]
    row=0
    all_amendement = []
    
    for f in os.listdir(folder):
        print("Lecture du fichier {}".format(f), end=',')
        current_file= open(folder+"/"+f)
        
        for line in current_file:
            all_amendement.append(normalize_text(line.strip()))
            if line.strip()[0:len(projet_loi)] == projet_loi:
                starts.append(row-5)
            row+=1
                
        current_file.close()
    return starts, all_amendement
        
starts_1, all_amendement_1 = lecture_amendement("ancienne_liasse/")
starts_2, all_amendement_2 = lecture_amendement("nouvelle_liasse/")



# In[53]:

def get_amendement_nb(x):
    try:
        return x[1].replace("N° ","")
    except:
        return "0"


# In[202]:

def get_amendement_text(x):
    start,end=0,0
    for i in range(0, len(x)):
        if x[i]==normalize_text("----------"):
            start=i+3
        if normalize_text(x[i][0:5])==normalize_text("EXPOS"):
            end=i
    candidate = x[start:end]
    res=[]
    for i in range(0, len(candidate)):
        if "Cet amendement est en cours" in candidate[i]:
            continue
        elif len(candidate[i]) < 2:
            continue
        else:
            res.append(candidate[i])
    return " ".join(res).replace("  ", " ")




# In[54]:

def get_amendements(starts, all_amendements, version):
    amendements_nb, amendements_txt, begins, ends = [],[],[], []
    for i in range(0, len(starts)):
        if i<len(starts)-1:
            begin=starts[i]
            end=starts[i+1]
        else:
            begin=starts[-1]
            end=len(all_amendements)
        #print(begin, end)
        current_amendement = all_amendements[begin:end]
        #print(starts[i], starts[i+1])
        amendements_nb.append(get_amendement_nb(current_amendement))
        amendements_txt.append(get_amendement_text(current_amendement))
        begins.append(begin)
        ends.append(end)
    return pd.DataFrame({'nb_'+version:amendements_nb, 'txt':amendements_txt, 'begin':begins, 'end':ends})


# In[204]:

df_amendements_1 = get_amendements(starts_1, all_amendement_1,'1')
print(len(df_amendements_1))
df_amendements_2 = get_amendements(starts_2, all_amendement_2,'2')
print(len(df_amendements_2))

Lecture du fichier amendements CAE liasse 1 .txt,Lecture du fichier 10_ après 15.txt,Lecture du fichier 11_art 15 bis à avant 16.txt,Lecture du fichier 12- art 16 à la fin.txt,Lecture du fichier 1_  avant 11 et 11 liasse 2.txt,Lecture du fichier 2 _ art 11- après 11.txt,Lecture du fichier 3_ 11 bis après 11 quindecies.txt,Lecture du fichier 4- art 11 sexdecies -après 12.txt,Lecture du fichier 5_ art 12 bis -après 13 ter.txt,Lecture du fichier 6_ après 13 ter- après 14.txt,Lecture du fichier 8_ 14 bis -après 14 undecies.txt,Lecture du fichier 9_ art 15.txt,874
1379


In [10]:
corresp={}
for i1, row1 in df_amendements_1.iterrows():
    if i1%10==0:
        print(i1, end=",")
    for i2,row2 in df_amendements_2.iterrows():
        l = get_levenshtein(row1.txt,row2.txt)
        if(l<0.3):
            corresp[row2.nb_2] = (row1.nb_1, row1.txt, row2.txt, l)
            continue

0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660,670,680,690,700,710,720,730,740,750,760,770,780,790,800,810,820,830,840,850,860,870,

In [32]:
excel1 = pd.read_excel('ancien_derouleur.xlsx')
excel2 = pd.read_excel('nouveau_derouleur_a_remplir2.xlsx')

In [13]:
def get_avis(df, nb):
    avis = df[df["N°"]==nb]["Avis SAJ "].values[0]
    return avis

def get_corresp_avis(corresp, nb):
    if nb in corresp:
        return get_avis(excel1, corresp[nb][0])
    return ""

def get_old_new_txt(corresp, nb):
    if nb in corresp:
        return (corresp[nb][1], corresp[nb][2], corresp[nb][3])
    return ("","","")

def get_text(df, nb):
    df_t = df[df.nb_2==nb]
    if len(df_t)>0:
        return df_t.txt.values[0]
    return ""

In [14]:
avis, old_txt, new_txt, difference_txt, txt2= [], [], [], [], []
for i2, row2 in excel2.iterrows():
    nb2 = str(row2["N°"])
    avis.append(get_corresp_avis(corresp, nb2))
    extra_cols = get_old_new_txt(corresp, nb2)
    old_txt.append(extra_cols[0])
    new_txt.append(extra_cols[1])
    txt2.append(get_text(df_amendements_2, nb2))
    difference_txt.append(extra_cols[2])

In [33]:
excel2["txt_amendement"] = txt2

excel2["Avis SAJ"]=avis
excel2["Amendent v1"]=old_txt
excel2["Amendent v2"]=new_txt
excel2["difference_amendement"]=difference_txt

In [22]:
df_tmp.head()

,txt_amendement,count,min
0,,1247,1
1,"""D’ici le 1er janvier 2030, les signes d’ident...",1,2470
2,"""I. Au plus tard le 1er janvier 2021,un décre...",1,2466
3,"""L'Etat se donne pour objectif à compter de 2...",1,2113
4,1° Après les mots « des structures » ajouter ...,1,1889


In [37]:
excel2["N°"] = excel2["N°"].apply(lambda x:str(x))
df_tmp = pd.DataFrame(excel2.groupby("txt_amendement")["N°"].agg(['count', "min"])).reset_index()


# In[195]:

def has_similar(df_tmp, txt):
    if len(txt)<3:
        return ""
    if df_tmp[df_tmp["txt_amendement"]==txt]["count"].values[0] > 1:
        return 'Amendement similaire au ' + str( df_tmp[df_tmp["txt_amendement"]==txt]["min"].values[0])
    return ""

In [38]:
similars = []
for i2, row2 in excel2.iterrows():
    similars.append(has_similar(df_tmp, row2.txt_amendement))

In [39]:
excel2["Amendent similaire ?"]=similars

In [40]:
excel2.to_excel('nouveau_derouleur_post_traitement.xlsx')